#### 보스턴  집값 예측 모델
- 데이터셋 : boston.scv
- 학습방법 : 지도학습 >> 회귀
- 피쳐/독립 : 13개 
- 타겟/종속 : 1개

[1] 데이터 준비

In [36]:
# 모듈로딩
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split


In [37]:
DATA_FILE=r'C:\Users\LG\Desktop\머신러닝\DATA\boston.csv'

In [38]:
#
dataDF= pd.read_csv(DATA_FILE)

In [39]:
dataDF.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [40]:
# e데이터 기본 정보 확인
dataDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


[2] 전처리

- [2-1] 테이터 정제

- 결측치, 중복값, 이상치, 컬럼별 고유값 추출로 이상 데이터 체크

- [2-2]  표준화& 정규화  ==> 진행여부에 따라서 성은의 변화는 경우에 따라 다름!
    * 정규분포 데이터셋을 기반으로 한 모델 ==> StandardScaler, MinMaxScaler, Log 변환
    * 피쳐의 값의 범위 차이를 줄이기 ==> 피쳐 스케일링, MinMaxScaler, RobsutScaler...
    * 범주형 피쳐==> 수치화  인코딩 OneHotEncoder, OrdinalEncoder
    * 문자열 타켓 ==> 정수 라벨인코딩 LabelEncoder

- [2-3] 피쳐와 타겟 분리

In [41]:
featureDF=dataDF.iloc[:,:-1]
targetSR= dataDF['MEDV']

In [42]:
print(f'featureDF: {featureDF.shape} targetSR : {targetSR.shape}')

featureDF: (506, 13) targetSR : (506,)


[3]학습 준비

[3-1]학슴용 데이터셋과 테스트용 데이터셋 분리

In [43]:
X_train, X_test, y_train, y_test =  train_test_split(featureDF,targetSR,random_state=10)


In [44]:
print(f'X')

X


[3-2] 학습용 데이터셋으로 스케일러 생성

In [45]:
###  - 수치 피쳐 값의 범위 차가 큼 => Scaling 진행
ssScaler=StandardScaler()
ssScaler.fit(X_train)

StandardScaler()

In [46]:
X_train_scaled =  ssScaler.transform(X_train)
X_test_scaled=  ssScaler.transform(X_test)

[4] 교차진행 ==> 교차검증으로 진행

In [47]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Ridge

In [48]:
#모델 인스턴스
ridge_model =Ridge(alpha= 1.0)#기본값 1.0

In [49]:
# ------ 모델의 성능을 좌우하는 Hyper-parmeter  제어 즉 튜닝
alpha_values=[0. ,1. ,10,100]

for value in alpha_values:

    #모델 인스턴스 생성
    ridge_model = Ridge(alpha=value)  #Lasso( alpha=value , max_inter=3)
    # 학습진행 
    # -cv :3 개 
    # - scoing : 'mean_squared_error, 'r2'
    # - return_train_score
    result = cross_validate(ridge_model,
                            X_train_scaled, y_train,
                            cv=3,
                            scoring=['neg_mean_squared_error', 'r2'],
                            return_train_score=True,
                            return_estimator=True)
    
    resultDF= pd.DataFrame(result) [['test_r2','train_r2']] # 가지고 오고싶은 컬럼[[]]

    result['diff']=abs(resultDF['test_r2']- resultDF['train_r2'])
    


    print(result['estimator'][0].coef_)
    # best_idx=resultDF['diff'].sort_values()[0]
    # print(best_idx)
    print(f'[Ridge(alpha= {value})]')
    print(resultDF, end =' \n\n')


[-1.41407793  1.56590993  0.15536906  0.65522098 -2.36200159  2.31948624
  0.1173831  -3.59071105  2.71475429 -2.33252925 -1.88390034  1.04036915
 -3.50250877]
[Ridge(alpha= 0.0)]
    test_r2  train_r2
0  0.747022  0.755720
1  0.756482  0.740082
2  0.680801  0.786156 

[-1.39035961  1.53043843  0.11109741  0.6621853  -2.29024619  2.34249774
  0.10030677 -3.52062389  2.57481444 -2.20749462 -1.86406784  1.03607796
 -3.48102887]
[Ridge(alpha= 1.0)]
    test_r2  train_r2
0  0.748283  0.755663
1  0.756292  0.740039
2  0.680991  0.786097 

[-1.23221033  1.29302258 -0.12737786  0.70280521 -1.80949922  2.48028701
 -0.00860666 -2.99831755  1.75466332 -1.51704375 -1.73434856  1.00368486
 -3.30809117]
[Ridge(alpha= 10)]
    test_r2  train_r2
0  0.753103  0.752474
1  0.755100  0.737457
2  0.677471  0.783225 

[-0.78141029  0.70910255 -0.46407849  0.72503917 -0.69294458  2.41757287
 -0.24148703 -1.21831206  0.28616643 -0.63423538 -1.31602563  0.78528977
 -2.39571659]
[Ridge(alpha= 100)]
    test_r2

In [50]:
resultDF=pd.DataFrame(result)

- 하이퍼파라미터 튜닝과 교차 검증을 동시에 진행

In [51]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

In [52]:
#Ridge 의 Hyper- parameter 값 설정
params= {'alpha':[0.,0.1,0.5,1.0],
         'max_iter' :[3,5]}

# 0.,3=> model    # 0.,5 => model
# 0.1,3=> model    # 0.1,5 => model
# 0.5,3=> model    # 0.5,5 => model
# 1.0,3=> model    # 1.0,5 => model
# ==> 8개의 Ridg   모델 생성

In [53]:
#인스턴스 생성
rModel=Ridge()

#GridSearchCV 인스턴스 생성
searchCV=GridSearchCV(rModel,params,cv=3,verbose=True, return_train_score= True)


In [54]:
#학습 진행
searchCV.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


GridSearchCV(cv=3, estimator=Ridge(),
             param_grid={'alpha': [0.0, 0.1, 0.5, 1.0], 'max_iter': [3, 5]},
             return_train_score=True, verbose=True)

In [55]:
#fit()진행후  모델 파라미터 확인
searchCV.best_params_

{'alpha': 1.0, 'max_iter': 3}

In [56]:
searchCV.best_index_

6

In [57]:
bestModel=searchCV.best_estimator_
bestModel

Ridge(max_iter=3)

In [58]:
resultDF=pd.DataFrame(searchCV.cv_results_)
resultDF

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.001480,3.643912e-04,0.000725,5.179975e-04,0.0,3,"{'alpha': 0.0, 'max_iter': 3}",0.747022,0.756482,0.680801,0.728101,0.033669,7,0.755720,0.740082,0.786156,0.760653,0.019131
1,0.000997,1.123916e-07,0.000664,4.695159e-04,0.0,5,"{'alpha': 0.0, 'max_iter': 5}",0.747022,0.756482,0.680801,0.728101,0.033669,7,0.755720,0.740082,0.786156,0.760653,0.019131
2,0.000664,4.697969e-04,0.000332,4.699093e-04,0.1,3,"{'alpha': 0.1, 'max_iter': 3}",0.747159,0.756462,0.680831,0.728151,0.033675,5,0.755720,0.740081,0.786156,0.760652,0.019131
3,0.000665,4.700217e-04,0.000332,4.696845e-04,0.1,5,"{'alpha': 0.1, 'max_iter': 5}",0.747159,0.756462,0.680831,0.728151,0.033675,5,0.755720,0.740081,0.786156,0.760652,0.019131
4,0.000666,4.708124e-04,0.000997,2.973602e-07,0.5,3,"{'alpha': 0.5, 'max_iter': 3}",0.747682,0.756385,0.680927,0.728331,0.033708,3,0.755705,0.740070,0.786141,0.760639,0.019129
5,0.000664,4.696845e-04,0.000665,4.700781e-04,0.5,5,"{'alpha': 0.5, 'max_iter': 5}",0.747682,0.756385,0.680927,0.728331,0.033708,3,0.755705,0.740070,0.786141,0.760639,0.019129
6,0.000997,4.495664e-07,0.000000,0.000000e+00,1.0,3,"{'alpha': 1.0, 'max_iter': 3}",0.748283,0.756292,0.680991,0.728522,0.033768,1,0.755663,0.740039,0.786097,0.760600,0.019124
7,0.000336,4.755288e-04,0.000747,5.378895e-04,1.0,5,"{'alpha': 1.0, 'max_iter': 5}",0.748283,0.756292,0.680991,0.728522,0.033768,1,0.755663,0.740039,0.786097,0.760600,0.019124
